In [307]:
# IMPORTANT CONDITION:
# Attributes of each node are the attributes that have more than one value in the current DataFarme.
# Because: they might be all the same but have different classifiers, hence producing bad splits
# 
# We shall not try to find a good split of a node that has the same classifier!!!
import pandas as pd
path='data.csv'
df=pd.read_csv(path)
#df=df[:20]
A=list(df.columns[:-1])
classifier=df.columns[-1]
D = df
default = 0
mytree= createNode(D, A, "Root", 0, classifier)
continueTree(mytree)
generalizationError(mytree, 0.5)

Generalization Error = 0.20078740157480315


0.20078740157480315

In [304]:
printDecisionTree(mytree)

Root
Level 0
Feature Sex 0
Gini 0.4724

Intermediate
Level 1
Feature Pclass 1 2
Gini 0.3844
*********
Intermediate
Level 1
Feature Pclass 1
Gini 0.3064

Intermediate
Level 2
Feature Embarked 0 1
Gini 0.1014
*********
Intermediate
Level 2
Feature Embarked 0 1
Gini 0.5
*********
Intermediate
Level 2
Feature Embarked 0
Gini 0.4656
*********
Intermediate
Level 2
Feature Embarked 0
Gini 0.2417

Intermediate
Level 3
Feature Pclass 1
Gini 0.03702
*********
Intermediate
Level 3
Feature Pclass 1
Gini 0.1295
*********
Intermediate
Level 3
Feature Embarked 0
Gini 0.4228
*********
Leaf
Level 3
Class 0
Gini 0.4688
*********
Leaf
Level 3
Class 0
Gini 0.4819
*********
Intermediate
Level 3
Feature Embarked 1
Gini 0.455
*********
Intermediate
Level 3
Feature Pclass 2
Gini 0.3503
*********
Intermediate
Level 3
Feature Embarked 1
Gini 0.2252

Intermediate
Level 4
Feature Embarked 0
Gini 0.04442
*********
Leaf
Level 4
Class 1
Gini 0
*********
Leaf
Level 4
Class 1
Gini 0.07986
*********
Leaf
Level 4
Class 

# FUNCTIONS

In [110]:
d=tree["tree"]
ds=d[d["Sex"]==1]
dd=d[d["Sex"]==0]
getVals(d, "Sex")

ddd=dd[dd["Pclass"]<3]
dds=dd[dd["Pclass"]>=3]
ddsd=dds[dds["Embarked"]<2]
ddss=dds[dds["Embarked"]>=2]

dsd=ds[ds["Pclass"]<3]
dss=ds[ds["Pclass"]>=3]

a=ddss
GINI(a, "Survived")
a

,Sex,Pclass,Embarked,Survived
2,0,3,2,1
8,0,3,2,1
10,0,3,2,1
14,0,3,2,0
18,0,3,2,0


In [111]:
atts=["Sex", "Pclass", "Embarked"]
d=ddss
ginis={}
for att in atts:
    vals = getVals(d, att)
    print(vals)
    for val in vals[1:]:
        ginis[(att ,val)]=GINIsplit(d, att, val, "Survived")
best = min(ginis, key=ginis.get)
best
ginis

[0]
[3]
[2]


ValueError: min() arg is an empty sequence

In [112]:
a,b=split(dds, "Pclass", 2)
a

,Sex,Pclass,Embarked,Survived


In [113]:
bestCandidate(dds, atts, "Survived")
GINI(ddss,"Survived")

0.48

In [114]:
allEqual(ddss, tree["attributes"])

True

In [115]:
a =[]
a ==[]

True

In [196]:
generalizationError(tree, 0.5)

Generalization Error = 0.275


(5.5, 3, 5)

In [177]:
myTree= createNode(D, A, "Root", 0, classifier)
continueTree(myTree)
generalizationError(myTree, 0.5)
myTree["left"]["right"]["type"]="leaf"
myTree["left"]["right"].pop("left")
myTree["left"]["right"].pop("right")
print(myTree["left"]["right"].keys())
#printDecisionTree(myTree)
generalizationError(myTree, 0.5)
#getLevels(myTree)
#getTreeOrdered(myTree)

Generalization Error = 0.275
dict_keys(['tree', 'attributes', 'type', 'level', 'classifier', 'min', 'default', 'gini', 'feature'])
Generalization Error = 0.125


(2.5, 1, 3)

In [278]:
a=[1,23,3]
aa = [1,2, a]
b =aa.copy()
c=b[2]
c[0]=2
print(aa,b,c)

[1, 2, [2, 23, 3]] [1, 2, [2, 23, 3]] [2, 23, 3]


In [281]:
generalizationError(mytree, 0.5)

Generalization Error = 0.275


0.275

In [288]:
import copy

root = copy.deepcopy(mytree)
walk(mytree, root, root)

Generalization Error = 0.275
Intermediate ('Embarked', 2) 
dict_keys(['tree', 'attributes', 'type', 'level', 'classifier', 'min', 'default', 'gini', 'feature', 'left', 'right'])
Generalization Error = 0.275
Intermediate ('Embarked', 2) 
dict_keys(['tree', 'attributes', 'type', 'level', 'classifier', 'min', 'default', 'gini', 'feature'])
Generalization Error = 0.275
Generalization Error = 0.25
improved!!!!!!!!!!!!
Generalization Error = 0.275
Intermediate ('Pclass', 3) 
dict_keys(['tree', 'attributes', 'type', 'level', 'classifier', 'min', 'default', 'gini', 'feature', 'left', 'right'])
Generalization Error = 0.275
Intermediate ('Pclass', 3) 
dict_keys(['tree', 'attributes', 'type', 'level', 'classifier', 'min', 'default', 'gini', 'feature'])
Generalization Error = 0.275
Generalization Error = 0.225
improved!!!!!!!!!!!!
Generalization Error = 0.275
Intermediate ('Pclass', 3) 
dict_keys(['tree', 'attributes', 'type', 'level', 'classifier', 'min', 'default', 'gini', 'feature', 'left', 'ri

In [289]:
def walk(originalTree, prunedTree, prunedRoot):
    tp = prunedTree["type"]
    if(tp!="Leaf"):
        walk(originalTree, prunedTree["left"], prunedRoot)
        walk(originalTree, prunedTree["right"], prunedRoot)
        prunedTree["type"] = "Leaf"
        left = prunedTree.pop("left")
        right= prunedTree.pop("right")
        print(tp, prunedTree["feature"],"\n" + str(prunedTree.keys()))
        originalError = generalizationError(originalTree, 0.5)
        if(generalizationError(prunedRoot, 0.5)<originalError):
            print("improved!!!!!!!!!!!!")
            return
        prunedTree["left"] =left
        prunedTree["right"]=right
        prunedTree["type"] =tp

In [315]:
mytree= createNode(D, A, "Root", 0, classifier)
continueTree(mytree)
nodesByLevel=getTreeOrdered(mytree)
levels= getLevels(mytree)[-1]
for level in range(levels,0,-1):
    for node in nodesByLevel[level]:
        if node["type"] != "Leaf":
            ierror=generalizationError(mytree,0.5)
            left=node.pop("left")
            right=node.pop("right")
            node["type"]="Leaf"
            error=generalizationError(mytree,0.5)
            if(error< ierror):
                print("Pruned leaves of a node at level", level)
            else:
                node["left"]=left
                node["right"]=right
                node["type"]="Intermediate"
generalizationError(mytree,0.5)

Generalization Error = 0.20078740157480315
178.5 170 17
Generalization Error = 0.20022497187851518
178.0 170 16
Pruned leaves of a node at level 4
Generalization Error = 0.20022497187851518
178.0 170 16
Generalization Error = 0.19853768278965128
176.5 169 15
Pruned leaves of a node at level 4
Generalization Error = 0.19853768278965128
176.5 169 15
Generalization Error = 0.19797525309336333
176.0 169 14
Pruned leaves of a node at level 4
Generalization Error = 0.19797525309336333
176.0 169 14
Generalization Error = 0.19741282339707536
175.5 169 13
Pruned leaves of a node at level 3
Generalization Error = 0.19741282339707536
175.5 169 13
Generalization Error = 0.1968503937007874
175.0 169 12
Pruned leaves of a node at level 3
Generalization Error = 0.1968503937007874
175.0 169 12
Generalization Error = 0.19628796400449944
174.5 169 11
Pruned leaves of a node at level 3
Generalization Error = 0.19628796400449944
174.5 169 11
Generalization Error = 0.19460067491563554
173.0 168 10
Pruned l

0.19122609673790777

In [320]:
nodesByLevel=getTreeOrdered(mytree)
levels= getLevels(mytree)[-1]
for level in range(levels,0,-1):
    for node in nodesByLevel[level]:
        if node["type"] != "Leaf":
            ierror=generalizationError(mytree,0.5)
            left=node.pop("left")
            right=node.pop("right")
            node["type"]="Leaf"
            error=generalizationError(mytree,0.5)
            if(error< ierror):
                print("Pruned leaves of a node at level", level)
            else:
                node["left"]=left
                node["right"]=right
                node["type"]="Intermediate"
generalizationError(mytree,0.5)

Generalization Error = 0.19122609673790777
170.0 168 4
Generalization Error = 0.21541057367829022
191.5 190 3
Generalization Error = 0.19122609673790777
170.0 168 4


0.19122609673790777

In [316]:
left=mytree["left"].pop("left")
right=mytree["left"].pop("right")
mytree["type"]="Leaf"
generalizationError(mytree,0.5)
mytree["left"]["left"]=left
mytree["left"]["right"]=right
mytree["left"]["type"]="Intermediate"
generalizationError(mytree,0.5)

Generalization Error = 0.12317210348706412
109.5 109 1
Generalization Error = 0.19122609673790777
170.0 168 4


0.19122609673790777

In [312]:
printDecisionTree(mytree)

Root
Level 0
Feature Sex 0
Gini 0.4724

Intermediate
Level 1
Feature Pclass 1 2
Gini 0.3844
*********
Leaf
Level 1
Class 0
Gini 0.3064

Leaf
Level 2
Class 1
Gini 0.1014
*********
Intermediate
Level 2
Feature Embarked 0 1
Gini 0.5

Leaf
Level 3
Class 1
Gini 0.4228
*********
Leaf
Level 3
Class 0
Gini 0.4688



In [226]:
def pruneTree(tree, alpha):
    prunedTree=dict(tree)
    for level in nodesByLevel[1:getLevels(tree)[-1]]:
        for node in level:
            generalizationError(node, alpha)
    return 0

In [58]:
def getLevels(tree):
    nodesByLevel= getTreeOrdered(tree)
    levels =[0]
    for a in nodesByLevel:
        for val in a:
            level= val["level"]
            if level not in levels:
                levels += [level]
    return levels

In [314]:
def generalizationError(tree, alpha):
    nodesByLevel= getTreeOrdered(tree)
    complexity=0
    mistakes=0
    for level in nodesByLevel[1:]:
        for node in level:
            if(node["type"]=="Leaf"):
                complexity+=1
                vals=node["tree"].T
                for row in node["tree"].T:
                    test = sum(node["tree"][node["classifier"]])/len(node["tree"]) > 0.5
                    cl = 1 if test else 0
                    if (len(node["tree"])<node["min"]): cl = node["default"]
                    if(vals[row][node["classifier"]]!=cl):
                        mistakes+=1
    error = mistakes + alpha * complexity
    print("Generalization Error =",error/len(tree["tree"]))
    print(error, mistakes, complexity)
    return error/len(tree["tree"])
    #return error, mistakes, complexity

In [18]:
def printDecisionTree(tree):
    printNode(tree)
    print()
    nodesByLevel= getTreeOrdered(tree)
    for level in nodesByLevel[1:]:
        if level == []:
            continue
        i=0
        for node in level:
            i+=1
            printNode(node)
            if (i != len(level)):
                print('*********')
        print()

In [19]:
def getTreeOrdered(tree):
    nodesByLevel=[[]]*len(tree["tree"].columns)*4 #recently edited!!!
    nodesByLevel[0] = [tree]
    getNextLevel(tree, nodesByLevel)
    return nodesByLevel

In [20]:
def getNextLevel(tree, result):
    if 'left' in tree.keys():
        result[tree['level']+1]= result[tree['level']+1] + [tree['left']]
        result[tree['level']+1]= result[tree['level']+1] + [tree['right']]
        getNextLevel(tree['left'],result)
        getNextLevel(tree['right'],result)

In [21]:
def printNode(node):
    nType = node["type"]
    nLevel= node["level"]
    nGini =node["gini"]
    print(nType)
    print("Level",nLevel)
    if nType in ("Root","Intermediate"):
        feature = node["feature"]
        print("Feature",feature[0],end=" ")
        for i in range(min(getVals(node["tree"],feature[0])),feature[1]):
            if (i != feature[1] - 1):
                print(i, end = " ")
            else:
                print(i, end = "")
        print()
    else:
        test = sum(node["tree"][node["classifier"]])/len(node["tree"]) > 0.5
        cl = 1 if test else 0
        if (len(node["tree"])<node["min"]): cl = node["default"]
        print("Class",cl)
    if(nGini!=0):
        print("Gini","{:.4}".format(nGini))
    else:
        print("Gini",0)

In [22]:
def continueTree(root):
    A = list(root["attributes"])
    D = root["tree"]
    classifier = root["classifier"]
    sameAttributes = allEqual(D, A)
    if len(D)<=root["min"] or root["gini"]==0 or sameAttributes:
        root["type"]= "Leaf"
    elif root["type"] != "Leaf":
        root["feature"]= [attribute, value] = bestCandidate(D, A, classifier) #don't call for best candidate if there are no new best candidates
        left,right = split(D, attribute, value)
        root["left"]  = createNode(left , A, "Intermediate", root["level"]+1, classifier)
        root["right"] = createNode(right, A, "Intermediate", root["level"]+1, classifier)
        continueTree(root["left"])
        continueTree(root["right"])

In [23]:
def allEqual(D, A):
    for att in A:
        iterator = D[att]
        iterator = iter(iterator)
        try:
            first = next(iterator)
        except StopIteration:
            return True
        answer = all(first == rest for rest in iterator)
        if answer == False:
            return answer
    return True

In [32]:
def bestCandidate(data, attributes, classifier):
    ginis={}
    for att in attributes:
        vals = getVals(data, att)
        for val in vals[1:]:
            ginis[(att ,val)]=GINIsplit(data, att, val, classifier)
    best = min(ginis, key=ginis.get)
    return best

In [25]:
def GINIsplit(data, attribute, value, classifier):
    """returns the Gini of the split if applied on the given attribute with the specified classifier, input is a dataframe"""
    [yes,no]=split(data, attribute, value)
    gini = (GINI(yes,classifier)*len(yes)+GINI(no, classifier)*len(no))/len(data)
    return gini

In [26]:
def split(data, attribute, value):
    """returns two nodes based on a given split (attribute < value), input is a dataframe"""
    yes = data[data[attribute] < value]
    no = data[data[attribute] >= value]
    return yes,no

In [27]:
def getVals(df, attribute):
    """returns the DISTINCT values of a specified attribute from a dataset"""
    vals = []
    for val in list(df[attribute]):
        if val not in vals:
            vals += [val]
    vals.sort()
    return vals

In [28]:
def GINI(data, classifier):
    """returns the Gini of the given node, based on the Class"""
    classVals=getVals(data,classifier)
    cl=data[classifier]
    sqSum = 0
    for val in classVals:
        prob= len(cl[cl==val])/len(data)
        sqSum += prob**2
    GINI= 1-sqSum
    return GINI

In [29]:
def createNode(D, A, nodeType, level, classifier, minNum=5,alpha=0.5, default = 1):
    node = {}
    node["tree"] = D
    node["attributes"] = A
    node["type"] = nodeType
    node["level"] = level
    node["classifier"] = classifier
    node["min"]=minNum
    node["default"]=default
    node["gini"]=float(GINI(D,classifier))
    return node